TEAM 17
SRN: PES1UG20CS403 G
     PES1UG20CS406 G
     PES1UG20CS427 G
     PES1UG20CS697 L

In [ ]:
! pip install -q kaggle
!pip install torch_geometric
!pip install node2vec

In [ ]:
# from google.colab import files

# files.upload()

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
!kaggle datasets download -d feyzazkefe/olid-dataset

In [ ]:
! mkdir data
! unzip /content/olid-dataset.zip -d data

In [ ]:
import pandas as pd
import networkx as nx
import torch
from torch_geometric.nn import GCNConv, Sequential
from torch_geometric.data import Data, DataLoader
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import numpy as np

In [ ]:
train_data=pd.read_csv('/content/data/olid-training-v1.0.tsv', delimiter='\t', encoding='utf-8')

In [ ]:
train_data

In [ ]:
train_data[['id','tweet','subtask_a']]

In [ ]:
# Construct the tweet node features and labels
X = train_data['tweet'].tolist() # tweet text
y_a = train_data['subtask_a'].tolist() # offensive/not offensive labels
y_b = train_data['subtask_b'].tolist() # targeted/at others labels
y_c = train_data['subtask_c'].tolist() # individual/group/other labels


In [ ]:
X

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# # Calculate the TF-IDF matrix
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(X)

# # Calculate the cosine similarity matrix
# cosine_sim_matrix = cosine_similarity(tfidf_matrix)

In [ ]:
# Define the hierarchical label structure
label_hierarchy = [
    [0, 1], #offensive/not offensive
    [2, 3], #directed/not directed
    [4, 5, 6] #directed to individual/group/other
]

In [ ]:
# Set the threshold for cosine similarity
cosine_sim_threshold = 0.7
edge_lists = []
node_lists = []
i = 0
level_labels = label_hierarchy[0]
# Loop over each level of the hierarchy
for i, level_labels in enumerate(label_hierarchy):

    level_nodes = []
    level_edges = []

    if(i == 0):
      col = 'subtask_a'
      node_attrs = train_data[['id','tweet',col]].set_index('id').to_dict('index')
      node_map = {}
      count = 0
      for key,value in node_attrs.items():
        if(value['subtask_a']!='OFF' and value['subtask_a']!='NOT'):
          print("nan")
        else:
          node_map[count] = key
          count += 1
    elif(i==1):
      col = 'subtask_b'
      node_attrs = train_data[['id','tweet',col]].set_index('id').to_dict('index')
      node_map = {}
      count = 0
      for key,value in node_attrs.items():
        if(value['subtask_b']!='UNT' and value['subtask_b']!='TIN'):
          print("nan")
        else:
          node_map[count] = key
          count += 1
    else:
      col = 'subtask_c'
      node_attrs = train_data[['id','tweet',col]].set_index('id').to_dict('index')
      node_map = {}
      count = 0
      for key,value in node_attrs.items():
        if(value['subtask_c']!='IND' and value['subtask_c']!='GRP'and value['subtask_c']!='OTH'):
          print("nan")
        else:
          node_map[count] = key
          count += 1

    node_idx = list(node_map.keys())
    #     # Calculate cosine similarity matrix for the nodes
    node_embeddings = tfidf_matrix[node_idx].toarray()
    similarity_matrix = cosine_similarity(node_embeddings)

    # Add edges to subgraph based on cosine similarity
    for u in range(len(node_idx)):
      for v in range(u+1, len(node_idx)):
          if similarity_matrix[node_idx[u], node_idx[v]] > cosine_sim_threshold:
              level_edges.append((node_map[u], node_map[v]))

    edge_lists.append(level_edges)
    node_lists.append(node_attrs)

In [ ]:
len(node_lists)

In [ ]:
len(edge_lists)

In [ ]:
G1 = nx.Graph()
for node_id, attrs in node_lists[0].items():
    G1.add_node(node_id, **attrs)
for source, target in edge_lists[0]:
    G1.add_edge(source,target)

In [ ]:
print(G1)

In [ ]:
G2 = nx.Graph()
for node_id, attrs in node_lists[1].items():
    G2.add_node(node_id, **attrs)
for source, target in edge_lists[1]:
    G2.add_edge(source,target)

In [ ]:
print(G2)

In [ ]:
G3 = nx.Graph()
for node_id, attrs in node_lists[2].items():
    G3.add_node(node_id, **attrs)
for source, target in edge_lists[2]:
    G3.add_edge(source,target)

In [ ]:
print(G3)

In [ ]:
from node2vec import Node2Vec
import pickle
node2vec = Node2Vec(G1, dimensions=32, walk_length=20, num_walks=100, p=1,q=2)
model = node2vec.fit(window=10, min_count=1)
# embeddings = {str(node_id): vec for node_id, vec in model.wv.vocab.items()}

In [ ]:
embeddings = {str(node_id): model.wv.get_vector(str(node_id)) for node_id in G1.nodes()}
with open('/content/data/embeddings1.pickle', 'wb') as f:
    pickle.dump(embeddings, f)
!zip -r data.zip /content/data
from google.colab import files
files.download("/content/data/embeddings1.pickle")

In [ ]:
node2vec = Node2Vec(G2, dimensions=32, walk_length=20, num_walks=100, p=1,q=2)
model = node2vec.fit(window=10, min_count=1)

In [ ]:
embeddings = {str(node_id): model.wv.get_vector(str(node_id)) for node_id in G2.nodes()}
with open('/content/data/embeddings2.pickle', 'wb') as f:
    pickle.dump(embeddings, f)
!zip -r data.zip /content/data
from google.colab import files
files.download("/content/data/embeddings2.pickle")

In [ ]:
node2vec = Node2Vec(G3, dimensions=32, walk_length=20, num_walks=100, p=1,q=2)
model = node2vec.fit(window=10, min_count=1)

In [ ]:
embeddings = {str(node_id): model.wv.get_vector(str(node_id)) for node_id in G3.nodes()}
with open('/content/data/embeddings3.pickle', 'wb') as f:
    pickle.dump(embeddings, f)
!zip -r data.zip /content/data
from google.colab import files
files.download("/content/data/embeddings3.pickle")

In [ ]:
with open('/content/data/embeddings1.pickle', 'rb') as f:
    embeddings = pickle.load(f)

In [ ]:
with open('/content/data/embeddings2.pickle', 'rb') as f:
    embeddings2 = pickle.load(f)

In [ ]:
with open('/content/data/embeddings3.pickle', 'rb') as f:
    embeddings3 = pickle.load(f)

In [ ]:
import torch_geometric
from torch_geometric.utils import from_networkx
# convert networkx graph to Pytorch geometric graph
data = from_networkx(G1)

In [ ]:
import torch_geometric
from torch_geometric.utils import from_networkx
# convert networkx graph to Pytorch geometric graph
data2 = from_networkx(G2)

In [ ]:
import torch_geometric
from torch_geometric.utils import from_networkx
# convert networkx graph to Pytorch geometric graph
data3 = from_networkx(G3)

In [ ]:
data

In [ ]:
data2

In [ ]:
data3

In [ ]:
node2vec_embeddings = torch.tensor([embeddings[str(node_id)] for node_id in list(G1.nodes)], dtype=torch.float)
node2vec_embeddings2 = torch.tensor([embeddings[str(node_id)] for node_id in list(G2.nodes)], dtype=torch.float)
node2vec_embeddings3 = torch.tensor([embeddings[str(node_id)] for node_id in list(G3.nodes)], dtype=torch.float)

In [ ]:
node2vec_embeddings

In [ ]:
data.x = node2vec_embeddings
data2.x = node2vec_embeddings2
data3.x = node2vec_embeddings3

In [ ]:
data.x.shape

In [ ]:
#CONVERTING BRANCHES TO INTEGER ENCODINGS
from sklearn.preprocessing import LabelEncoder

# Example list of classes
class_list = data.subtask_a
class_list2 = data2.subtask_b
class_list3 = data3.subtask_c

# Initialize label encoder and fit to classes
label_encoder = LabelEncoder()
label_encoder.fit(class_list)
label_encoder2 = LabelEncoder()
label_encoder2.fit(class_list2)
label_encoder3 = LabelEncoder()
label_encoder3.fit(class_list3)

# Transform classes to numbered classes
class_nums = label_encoder.transform(class_list)
class_nums2 = label_encoder2.transform(class_list2)
class_nums3 = label_encoder3.transform(class_list3)

print(len(class_nums))  # Output: [0 1 2 0 2]

In [ ]:
class_nums

In [ ]:
#GETTING DATA LABELS Y AS THE CLASS LABELS
data.y = torch.tensor(class_nums)
data2.y = torch.tensor(class_nums2)
data3.y = torch.tensor(class_nums3)

In [ ]:
print(set(data.subtask_a))
print(set(data2.subtask_b))
print(set(data3.subtask_c))

In [ ]:
data.x

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self,level):
        super().__init__()
        torch.manual_seed(1234)
        # self.conv1 = GCNConv(data.x.shape[1], 16)  # total number of feature columns
        self.conv2 = GCNConv(16, 8)
        self.conv3 = GCNConv(8, 4)
        self.conv4 = GCNConv(4, 2)
        if(level == 0):
          self.conv1 = GCNConv(data.x.shape[1], 16)  # total number of feature columns
          self.classifier = Linear(2, len(set(data.subtask_a)))  # total number of classification branches
        elif(level == 1):
          self.conv1 = GCNConv(data2.x.shape[1], 16)  # total number of feature columns
          self.classifier = Linear(2, len(set(data2.subtask_b)))  # total number of classification branches
        elif(level == 2):
          self.conv1 = GCNConv(data3.x.shape[1], 16)  # total number of feature columns
          self.classifier = Linear(2, len(set(data3.subtask_c)))  # total number of classification branches
    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()
        h = self.conv4(h, edge_index)
        h = h.tanh()  # Final GNN embedding space.

        # Apply a final (linear) classifier.
        out = self.classifier(h)
        # out = out[data.test_mask]

        return out, h


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def visualize_graph(G, color):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                     node_color=color, cmap="Set2")
    plt.show()


def visualize_embedding(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    h = h.detach().cpu().numpy()
    plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
    if epoch is not None and loss is not None:
        plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    plt.show()

In [ ]:
#TESTING MODEL
model = GCN(0)

_, h = model(data.x, data.edge_index)
print(f'Embedding shape: {list(h.shape)}')

visualize_embedding(h, color=class_nums)

In [ ]:
#TESTING MODEL
model = GCN(1)

_, h = model(data2.x, data2.edge_index)
print(f'Embedding shape: {list(h.shape)}')

visualize_embedding(h, color=class_nums2)

In [ ]:
#TESTING MODEL
model = GCN(2)

_, h = model(data3.x, data3.edge_index)
print(f'Embedding shape: {list(h.shape)}')

visualize_embedding(h, color=class_nums3)

In [ ]:
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader
from sklearn.metrics import accuracy_score


# Define train and test masks
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
train_mask[:int(0.8*data.num_nodes)] = 1
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask[int(0.8*data.num_nodes):] = 1


In [ ]:
import time
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 430})'''))

model = GCN(0)
criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)  # Define optimizer with higher learning rate.

def train(data):
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out, h = model(data.x, data.edge_index)  # Perform a single forward pass.
    loss = criterion(out[train_mask], data.y[train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss, h

def test(data):
    model.eval()
    out, h = model(data.x, data.edge_index)
    y_pred = out.argmax(dim=-1)
    y_true = data.y[test_mask]
    acc = accuracy_score(y_true.cpu().numpy(), y_pred[test_mask].cpu().numpy())
    return acc

for epoch in range(1000):
    loss, h = train(data)
    if epoch % 10 == 0:
        visualize_embedding(h, color=data.y, epoch=epoch, loss=loss)
        time.sleep(0.3)

acc = test(data)
print(f'Test Accuracy: {acc}')

In [ ]:
model = GCN(1)
criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)  # Define optimizer with higher learning rate.
for epoch in range(1000):
    loss, h = train(data2)
    if epoch % 10 == 0:
        visualize_embedding(h, color=data2.y, epoch=epoch, loss=loss)
        time.sleep(0.3)

acc = test(data2)
print(f'Test Accuracy: {acc}')

In [ ]:
model = GCN(2)
criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)  # Define optimizer with higher learning rate.
for epoch in range(1000):
    loss, h = train(data3)
    if epoch % 10 == 0:
        visualize_embedding(h, color=data3.y, epoch=epoch, loss=loss)
        time.sleep(0.3)

acc = test(data3)
print(f'Test Accuracy: {acc}')